# Conversation Summary Buffer
ConversationSummaryBufferMemory combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

Let's first walk through how to use the utilities.

## Using memory with LLM

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI

llm = OpenAI()

In [ ]:
from pprint import pprint

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)

In [ ]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [ ]:
pprint(memory.prompt)

In [ ]:
memory.buffer

In [ ]:
memory.load_memory_variables({})

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [ ]:
memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [ ]:
from pprint import pprint
pprint(memory.load_memory_variables({}))

We can also utilize the predict_new_summary method directly.

In [ ]:
messages = memory.chat_memory.messages
print(messages)

In [ ]:

previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

## Using in a chain
Let's walk through an example, again setting verbose=True so we can see the prompt.

In [ ]:
# from langchain.chains import ConversationChain
from langchain.chains.conversation.base import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    # verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")

In [ ]:
conversation_with_summary.predict(input="Just working on writing some documentation!")

In [ ]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")

In [ ]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(
    input="Haha you are wrong, although a lot of people confuse it for that"
)